<a href="https://colab.research.google.com/github/hucarlos08/GEO-ML/blob/main/MissingValues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Import necessary libraries
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

import pandas as pd


# Fetch data using fetch_openml
data = fetch_openml(data_id=45068, as_frame=True, parser='pandas')

# Separate features and target
X = data.data
y = data.target

# Combine them into a single dataframe
df = pd.concat([X, y], axis=1)

# Drop the rows where at least one element is missing.
df = df.dropna()

# Separate features and target again
X = df.drop([y.name], axis=1)
y = df[y.name]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical and numerical columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns



# Define preprocessing for numerical columns (standard scaler)
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # or median, most_frequent
    ('scaler', StandardScaler())
])

# Define preprocessing for categorical columns (one-hot encoding)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # or constant
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Append classifier to preprocessing pipeline
# Now we have a full prediction pipeline
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Train the model
clf.fit(X_train, y_train)

# Predict the classes
y_pred = clf.predict(X_test)

# Evaluate the model
print("Model accuracy: %.3f" % accuracy_score(y_test, y_pred))


Model accuracy: 0.851
